### Feature engineering - CategoryEncoding - Indexers
CategoryEncoding with sklearn on the Titanic dataset 

*** 
#### Environment
`conda activate sklearn-env`

***
#### Goals
- Replace categorical and discrete columns with categorical elements corespondng to each value fomr dataset

***
#### References
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html

#### Basic python imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

#### Dataset load using sklearn API from https://www.openml.org site

https://www.openml.org/d/40945

In [ ]:
from sklearn.datasets import fetch_openml

# Load data from https://www.openml.org/d/40945
raw_dataset = fetch_openml("titanic", version=1, as_frame=True).frame
dataset = raw_dataset.copy()
dataset.head(10)

In [ ]:
dataset.drop(['boat', 'body', 'home.dest'],  axis=1, inplace=True)
dataset = dataset.dropna().copy()

### Verify categorical /discrete fields in dataset

Notice:
- `pclass` discrete fiels having 3 distinct values.
- `sex` and `embarked` fields - categorical

In [ ]:
dataset.info()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder().fit(dataset[['pclass', 'sex', 'embarked']])
encoder.categories_

In [ ]:
dataset[['pclass', 'sex', 'embarked']] = encoder.transform(dataset[['pclass', 'sex', 'embarked']])

### Notice changed column type for 'pclass', 'sex', 'embarked'

In [ ]:
dataset.sample(20)